In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

# import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
# pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'metrics')
sys.path.append(source_path)
import matplotlib.pyplot as plt

from dl import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_train_data
)
from metrics import calc_spread_return_sharpe
print(torch.__version__)

1.11.0


In [2]:
print(torch.version.cuda), torch.cuda.is_available(), torch.cuda.device_count()

11.3


(None, True, 1)

##  Get Data and train a Neural Network

# Get the unique security codes

In [3]:
train_df = get_train_data()
print('Unique adjustment factor:', train_df['AdjustmentFactor'].unique())
print(train_df['AdjustmentFactor'])
train_df.head()

Unique adjustment factor: [ 1.          0.5         5.          0.33333333  0.83333333  0.25
  0.90909091  0.1        10.          0.2         0.95238095  2.
  0.14285714  0.66666667  0.86956522  0.76923077  0.8         4.
 20.        ]
Date
2017-01-04    1.0
2017-01-04    1.0
2017-01-04    1.0
2017-01-04    1.0
2017-01-04    1.0
             ... 
2021-12-03    1.0
2021-12-03    1.0
2021-12-03    1.0
2021-12-03    1.0
2021-12-03    1.0
Name: AdjustmentFactor, Length: 2332531, dtype: float64


,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
Date,,,,,,,,,,,
2017-01-04,20170104_1301,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
2017-01-04,20170104_1332,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2017-01-04,20170104_1333,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
2017-01-04,20170104_1376,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
2017-01-04,20170104_1377,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


# Create model

In [4]:
TS_IN_FEATURES = 1

CAT_FEATURES = 4 #cat.shape[1]

print('CAT_FEATURES:', CAT_FEATURES)
EMBEDDING_DIM = 200
NO_EMBEDDING = 2000 #2 * len(df_train_cat)
print('NO_EMBEDDING:', NO_EMBEDDING)

model = NeuralNetwork(
    in_features=TS_IN_FEATURES, 
    units=1024,
    out_features=1, 
    categorical_dim=CAT_FEATURES,
    no_embedding=NO_EMBEDDING, 
    emb_dim=EMBEDDING_DIM,
    n_blocks=8,
    n_stacks=4,
    dropout=0.1
)

print(model)

CAT_FEATURES: 4
NO_EMBEDDING: 2000
NeuralNetwork(
  (embedding): Embedding(2000, 200)
  (embedding_to_hidden): Linear(in_features=200, out_features=1024, bias=True)
  (embedding_output): Linear(in_features=1024, out_features=1, bias=True)
  (cont_input): Linear(in_features=1, out_features=1024, bias=True)
  (hidden_layer): Linear(in_features=1028, out_features=1028, bias=True)
  (output_layer): Linear(in_features=1028, out_features=1, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (stacks): ModuleList(
    (0): NeuralStack(
      (stacks): ModuleList(
        (0): NeuralBlock(
          (dropout): Dropout(p=0.1, inplace=False)
          (layer1): Linear(in_features=1028, out_features=1028, bias=True)
          (layer2): Linear(in_features=1028, out_features=1028, bias=True)
          (layer3): Linear(in_features=1028, out_features=1028, bias=True)
          (layer4): Linear(in_features=1028, out_features=1028, bias=True)
          (layer5): Linear(in_features=1028, out_feature

In [5]:
# Loop throug each stock

In [ ]:
%%time
stocks = train_df['SecuritiesCode'].unique()
count = 0
for s in stocks:
    print(f'Start training for stock: {s}')
    train_loader, val_dataloader = None, None
    if count > 5:
        break
    train_dataloader, val_dataloader = dataloader_by_stock(train_df, s)

    torch.autograd.set_detect_anomaly(True)
    trainer = Trainer(model, lr=1.3e-9)
    trainer.fit_epochs(
        train_dataloader, 
        val_dataloader, 
        use_cyclic_lr=True, 
        x_cat=True, 
        epochs=20
    )
    print('#' * 20)
    print()
    count += 1
    
    

Start training for stock: 1301


C:\Users\gilbe\Documents\TokyoStockEx\notebooks\..\preprocessing\preprocess.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat['RowId'] = np.squeeze(enc.fit_transform(df['RowId'].to_numpy().reshape(-1, 1)))


continuos shape: (1202, 8)  categorical shape: (1202, 4)
Using cuda-device
Epoch: <<< 0 >>>


C:\Users\gilbe\anaconda3\envs\tokyo\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Copy.cpp:239.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Train metrics: {'mse': 0.010847826488316059, 'mae': 0.09627405554056168}
Validation metrics: {'mse': 0.006587217561900616, 'mae': 0.06131988391280174}
Average train loss: 0.0006432488560676574 | Average val loss: 0.0062713585793972015
.................... End of epoch 0 ....................
Epoch: <<< 1 >>>
